In [3]:
# Simple question: what is the critical value that will dictate whether I should knock on the first turn or not?

# In my mind, only two things factor into this decision:

# 1) What will my score be after my turn ends?
# 2) How many people are between me and the dealer? Do I go first (meaning 0 turns other than the last turns after the knock) or do I go last (meaning 3 extra turns before I knock..)?

# With a very simple "AI" - one that makes the best decision possible based on the cards in the hand of the user, we should be able to simulate the winning likelihood given a knock.

In [5]:
!pip install deck_of_cards

  Using cached deck_of_cards-0.0.10-py3-none-any.whl.metadata (2.4 kB)
Using cached deck_of_cards-0.0.10-py3-none-any.whl (7.0 kB)


In [359]:
# 1) First, we need to codify how a person will play given a knock has happened. In this first case, let us assume that I am right after the dealer (simplest case)

# 1a) A user first needs to draw: if the card that has been discarded fits into a run I have, or would create a run, always take it. If not, draw from the other pile
# 1b) Now it is time to lay cards down: lay your own runs down and add cards to other runs on the table if they are there
# 1c) Finally, we discard. Always discard the largest card in your hand

# based on this we need a few functions

from deck_of_cards import deck_of_cards
from collections import defaultdict

def card_value(card):
    """
    Numeric value used for discard logic.
    """
    rank = card.rank
    return int(rank)

def hand_value(hand):
    sums = 0
    for i in hand:
        true_val = min(i.rank,10) # either takes the value, or 10 because that is what face cards are worth
        sums += true_val

    return sums

def sort_hand(hand):
    return sorted(hand, key=card_value)

def show_hand(hand):
    for i in hand:
        print(i.rank,i.suit)

def deal():
    """
    Deals 7 cards to 4 players.
    Returns players, stock, discard
    """
    print("Dealing...")
    deck = deck_of_cards.DeckOfCards()
    deck.shuffle_deck()

    players = {i: [] for i in range(4)}
    for _ in range(7):
        for i in players:
            players[i].append(deck.give_random_card())

    discard_pile = [deck.give_random_card()] # initial card put there
    print("Top card after the deal is: ", discard_pile[0].rank, " of ", discard_pile[0].suit)
    stock = deck
    print()

    return players, stock, discard_pile


def draw(hand, stock, discard_pile):
    """
    Take from discard if it helps a run, else draw from stock
    """
    top_discard = discard_pile[-1]
    needed = runs(hand)

    discard_key = (card_value(top_discard), top_discard.suit)

    if discard_key in needed:
        print("Taking from the discard pile.")
        hand.append(discard_pile.pop())
    else:
        print("Taking from the deck.")
        hand.append(stock.give_random_card())

    return hand


def find_runs(hand):
    """
    Returns list of runs (each run is a list of cards)
    """
    by_suit = defaultdict(list)
    for c in hand:
        by_suit[c.suit].append(c)

    runs = []
    for suit, cards in by_suit.items():
        cards = sorted(cards, key=card_value)
        current = [cards[0]]

        for c in cards[1:]:
            if card_value(c) == card_value(current[-1]) + 1:
                current.append(c)
            else:
                if len(current) >= 3:
                    runs.append(current)
                current = [c]

        if len(current) >= 3:
            runs.append(current)
    print("Available runs: ", runs)
    return runs


def runs(hand):
    """
    Returns a set of cards (rank, suit tuples) that would extend runs
    """
    needed = set()
    for run in find_runs(hand):
        low = card_value(run[0])
        high = card_value(run[-1])
        suit = run[0].suit

        if low > 1:
            needed.add((low - 1, suit))
        if high < 13:
            needed.add((high + 1, suit))

    return needed

def give_table_runs(hand, table_runs):
    """
    Lays down new runs and adds cards to existing table runs
    """
    # Lay new runs
    new_runs = find_runs(hand)
    for run in new_runs:
        table_runs.append(run)
        for c in run:
            hand.remove(c)

    # Add to existing table runs
    for run in table_runs:
        suit = run[0].suit
        low = card_value(run[0])
        high = card_value(run[-1])

        added = True
        while added:
            added = False
            for c in hand[:]:
                v = card_value(c)
                if c.suit == suit and (v == low - 1 or v == high + 1):
                    if v == low - 1:
                        run.insert(0, c)
                        low -= 1
                    else:
                        run.append(c)
                        high += 1
                    hand.remove(c)
                    added = True

    return hand, table_runs


def discard(hand, discard_pile):
    """
    Discard the largest-value card
    """
    card = max(hand, key=card_value)
    hand.remove(card)
    discard_pile.append(card)

    show_hand(hand)

    return hand


In [380]:
players, stock, discard_pile = deal()
table_runs = []

reed = players[1]  # first after dealer

reed = draw(reed, stock, discard_pile)
reed, table_runs = give_table_runs(reed, table_runs)
reed = discard(reed, discard_pile)
hand_value(reed)

Dealing...
Top card after the deal is:  7  of  0

Available runs:  []
Taking from the deck.
Available runs:  []
7 2
5 0
11 3
11 1
11 0
1 1
6 0


49

In [381]:
ty = players[2]  # second after dealer

ty = draw(ty, stock, discard_pile)
ty, table_runs = give_table_runs(ty, table_runs)
discard(ty, discard_pile)
hand_value(ty)

Available runs:  []
Taking from the deck.
Available runs:  []
8 2
8 1
10 0
9 1
1 0
2 0
9 2


47

In [382]:
syd = players[3]  # third after dealer

syd = draw(syd, stock, discard_pile)
syd, table_runs = give_table_runs(syd, table_runs)
discard(syd, discard_pile)
hand_value(syd)

Available runs:  [[<deck_of_cards.deck_of_cards.Card object at 0x726c343198b0>, <deck_of_cards.deck_of_cards.Card object at 0x726c3431afc0>, <deck_of_cards.deck_of_cards.Card object at 0x726c341fb710>]]
Taking from the deck.
Available runs:  [[<deck_of_cards.deck_of_cards.Card object at 0x726c343198b0>, <deck_of_cards.deck_of_cards.Card object at 0x726c3431afc0>, <deck_of_cards.deck_of_cards.Card object at 0x726c341fb710>]]
1 3
2 2
4 2
6 1


13

In [383]:
dealer = players[0]

dealer = draw(dealer, stock, discard_pile)
dealer, table_runs = give_table_runs(dealer, table_runs)
discard(dealer, discard_pile)
hand_value(dealer)

Available runs:  []
Taking from the deck.
Available runs:  []
10 1
3 2
12 2
3 0
5 1
10 2
9 0


50

In [384]:
# awesome.. now we just need this in a loop somewhere where we can pull out details of who would win, and what the score required would be to win..